# 🧩 `parse_markdown_examples_to_json(text: str)`

Парсит текст с примерами, содержащими блоки `**Input:**` и `**Output:**`, и возвращает их в виде словаря. Этот формат используется у LeetCode.

---

## 📥 Аргументы

| Параметр | Тип   | Описание                             |
|----------|-------|--------------------------------------|
| `text`   | `str` | Текст, содержащий примеры в формате: |
| -        |       | `**Example N:**`                     |
| -        |       | `**Input:** <входные данные>`        |
| -        |       | `**Output:** <выходные данные>`      |

---

## 📤 Возвращает

Тип: `dict`

Структура возвращаемого словаря:

```json
{
  "examples": [
    {
      "input": {...},     // Входные данные (dict)
      "output": ...       // Выходные данные (любой тип)
    }
  ]
}
```

---


# 🔻 Вставьте пример для конвертирования в JSON

In [6]:
example_block = """
**Example 1:**

**Input:** nums = [-1,0,1,2,-1,-4]
**Output:** [ [-1,-1,2],[-1,0,1] ]
**Explanation:**
nums[0] + nums[1] + nums[2] = (-1) + 0 + 1 = 0.
nums[1] + nums[2] + nums[4] = 0 + 1 + (-1) = 0.
nums[0] + nums[3] + nums[4] = (-1) + 2 + (-1) = 0.
The distinct triplets are [-1,0,1] and [-1,-1,2].
Notice that the order of the output and the order of the triplets does not matter.

**Example 2:**

**Input:** nums = [0,1,1]
**Output:** []
**Explanation:** The only possible triplet does not sum up to 0.

**Example 3:**

**Input:** nums = [0,0,0]
**Output:** [ [0,0,0] ]
**Explanation:** The only possible triplet sums up to 0.
"""

## 1. Функция-обработчик

In [7]:
import re
import ast
import json

def parse_markdown_examples_to_json(text: str):
    examples = []
    raw_blocks = re.split(r"\*\*Example \d+:\*\*", text)

    for idx, block in enumerate(raw_blocks, 1):
        prefix = f"Example {idx}"

        # Попытка найти input
        input_lines = re.findall(r"\*\*Input:\*\*\s*(.*?)(?=\n\*\*|$)", block, re.DOTALL)
        if not input_lines:
            continue

        input_lines = [line.strip().rstrip(",") for line in input_lines[0].strip().splitlines() if line.strip()]
        input_text = "\n".join(input_lines)

        # Заменяем JSON-значения на Python-эквиваленты
        input_text = re.sub(r"\bnull\b", "None", input_text)
        input_text = re.sub(r"\btrue\b", "True", input_text)
        input_text = re.sub(r"\bfalse\b", "False", input_text)

        try:
            if len(input_lines) == 1 and '=' in input_lines[0]:
                # Пример: root = [...]
                input_expr = "{" + re.sub(r"(\w+)\s*=", r'"\1":', input_text) + "}"
                input_data = ast.literal_eval(input_expr)
            else:
                # Пример: ["MyQueue"], [[], []]
                input_data = {"args": [ast.literal_eval(line) for line in input_lines]}
        except Exception as e:
            print(f"{prefix} ⚠ Ошибка в input: {e}")
            continue

        # Попытка найти output
        output_match = re.search(r"\*\*Output:\*\*\s*(.*?)\s*(?:\n|$)", block)
        if not output_match:
            continue

        output_text = output_match.group(1).strip()
        output_text = output_text.replace("null", "None").replace("true", "True").replace("false", "False")

        try:
            output_data = ast.literal_eval(output_text)
        except Exception as e:
            print(f"{prefix} ⚠ Ошибка в output: {e}")
            continue

        examples.append({
            "input": input_data,
            "output": output_data
        })

    return {"examples": examples}


## 2. Парсим примеры из текста

In [8]:
parsed = parse_markdown_examples_to_json(example_block)
print(parsed)

{'examples': [{'input': {'nums': [-1, 0, 1, 2, -1, -4]}, 'output': [[-1, -1, 2], [-1, 0, 1]]}, {'input': {'nums': [0, 1, 1]}, 'output': []}, {'input': {'nums': [0, 0, 0]}, 'output': [[0, 0, 0]]}]}


## 3. Преобразуем результат в JSON-строку с красивым форматированием

In [9]:
pretty = json.dumps(parsed, indent=2, ensure_ascii=False)
print(pretty)

{
  "examples": [
    {
      "input": {
        "nums": [
          -1,
          0,
          1,
          2,
          -1,
          -4
        ]
      },
      "output": [
        [
          -1,
          -1,
          2
        ],
        [
          -1,
          0,
          1
        ]
      ]
    },
    {
      "input": {
        "nums": [
          0,
          1,
          1
        ]
      },
      "output": []
    },
    {
      "input": {
        "nums": [
          0,
          0,
          0
        ]
      },
      "output": [
        [
          0,
          0,
          0
        ]
      ]
    }
  ]
}


## 4. Убираем лишние пробелы и переносы строк внутри массивов

In [10]:
pretty = re.sub(r'\[\s+([^\[\]]+?)\s+\]', lambda m: '[' + m.group(1).replace('\n', '').replace(' ', '') + ']', pretty)
print(pretty)

{
  "examples": [
    {
      "input": {
        "nums": [-1,0,1,2,-1,-4]
      },
      "output": [
        [-1,-1,2],
        [-1,0,1]
      ]
    },
    {
      "input": {
        "nums": [0,1,1]
      },
      "output": []
    },
    {
      "input": {
        "nums": [0,0,0]
      },
      "output": [
        [0,0,0]
      ]
    }
  ]
}


# 🔺 Заберите результат в буфер обмена